In [ ]:
# Data download and unzipping
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c

!mkdir stage1_train stage1_test

!unzip stage1_train.zip -d stage1_train/
!unzip stage1_test.zip -d stage1_test/

In [ ]:
from google.colab import files
# upload .py files
uploaded = files.upload()

In [ ]:
!pip install keras --upgrade

In [ ]:
import os
import random
import sys
import warnings
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize

from keras.models import Model, load_model
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Setting seed for reproducability
SEED = 42
np.random.seed(SEED)
%matplotlib inline

In [ ]:
from data_utils import read_train_data, read_test_data, rle_encoding, mask_to_rle

In [ ]:
# get train_data
X_train,Y_train = read_train_data()
# get test_data
test_img,test_img_sizes = read_test_data()

train_img, val_img, train_mask, val_mask = train_test_split(X_train, Y_train, test_size=0.25, random_state=SEED)

train_size = len(train_img)
val_size = len(val_img)

In [ ]:
# get u_net model
u_net = get_unet(OPTIMIZER=optimizers.Adam(lr=LEARNING_RATE))
# u_net.summary()

### Training

In [ ]:
print("\nTraining...")
filepath="best_model.h5" # save best model only
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpointer = ModelCheckpoint(filepath, monitor='val_my_iou_metric', 
                               verbose=1, save_best_only=True, mode='max')
results = u_net.fit(x=train_img, y=train_mask, validation_data=(val_img, val_mask), 
                   batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
                   callbacks=[earlystopper, checkpointer])

### Plot training history

In [ ]:
loss = results.history['loss']
val_loss = result.history['val_loss']
acc = result.history['my_iou_metric']
val_acc = result.history['val_my_iou_metric']
plt.figure()
plt.plot(np.arange(len(loss)), np.array(loss), label='train loss')
plt.plot(np.arange(len(val_loss)), np.array(val_loss), label='val loss')
plt.legend(loc=1)

plt.figure()
plt.plot(np.arange(len(acc)), np.array(acc), label='train iou')
plt.plot(np.arange(len(val_acc)), np.array(val_acc), label='val iou')
plt.legend(loc=4)

### Load best model and evaluate

In [ ]:
best_model = load_model('weights.best.hdf5', custom_objects={'my_iou_metric': my_iou_metric})
print('Predicting...')
train_pred = best_model.predict(train_img, verbose=1)
val_pred = best_model.predict(val_img, verbose=1)
test_pred = best_model.predict(test_img, verbose=1)
train_iou = iou_metric_batch(train_mask, train_pred)
val_iou = iou_metric_batch(val_mask, val_pred)
print('Best train IOU = ', train_iou)
print('Best val IOU = ', val_iou)

### Visualizations

In [ ]:
idx = np.random.choice(range(len(val_img)))
plt.figure(figsize=(12,12))
plt.subplot(131)
imshow(val_img[idx])
plt.title("val image")
plt.subplot(132)
imshow(np.squeeze(val_mask[idx]))
plt.title("val mask")
plt.subplot(133)
imshow(np.squeeze(pred_val[idx] > 0.5))
plt.title("val predictions")
plt.show()

In [ ]:
idx = np.random.choice(range(len(test_img)))
plt.figure(figsize=(12,12))
plt.subplot(121)
imshow(test_img[idx])
plt.title("Image")
plt.subplot(122)
imshow(np.squeeze(pred_test[idx] > 0.5))
plt.title("Mask Prediction")
plt.show()

### Submission

In [ ]:
# Create list of upsampled test masks
test_mask_upsampled = []
for i in range(len(pred_test)):
    test_mask_upsampled.append(resize(np.squeeze(pred_test[i]),
                                       (test_img_sizes[i][0],test_img_sizes[i][1]), 
                                       mode='constant', preserve_range=True))

In [1]:
test_ids,rles = mask_to_rle(test_mask_upsampled)

NameError: name 'mask_to_rle' is not defined

In [ ]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))

sub.to_csv('sub-dsbowl2018.csv', index=False)

In [ ]:
# Code to download files from Google colab

from google.colab import files
files.download('sub-dsbowl2018.csv')
files.download('best_model.h5')